In [ ]:
!pip install ortools

In [ ]:
from ortools.linear_solver import pywraplp





def main():
  M = 9*(10**9)
  for count in range(3,4):
    tmp = []
    with open(r"/content/drive/MyDrive/Tối ưu/data/DataSet{}/input.txt".format(count),"r") as f:
      
      for i in f.readlines():
        i = i.replace("\n","")
        i = i.rstrip(" ").split(" ")
        tmp.append([int(j) for j in i])
    f.close()

    distanceMatrix = tmp[3:]
    hanhkhach = tmp[0][0]
    goihang = tmp[0][1]
    vehicleNumber = tmp[0][2]
    khoiLuongGoiHang = tmp[1]
    khoiLuongToiDaMoiXe = tmp[2]
    # Create the mip solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver('SCIP')
    infinity = solver.infinity()

    # Variables.

    x = []
    for i in range(2*(hanhkhach+goihang) +1):
      x.append([])
      for j in range(2*(hanhkhach+goihang) +1):
          x[i].append([])
          for k in range(vehicleNumber):
            x[i][j].append(solver.IntVar(
                0, 1, 'x[{}][{}][{}]'.format(i, j, k)))
    y = []
    for i in range(hanhkhach+goihang):
      y.append([])
      for k in range(vehicleNumber):
          y[i].append(solver.IntVar(0, 1, 'y[{}][{}]'.format(i, k)))

    # z = []
    # # x = [] # x[i][k] = 1 iff node i is in the position k in a permutation
    # for v in range(goihang):
    #   z.append([])
    #   for k in range(vehicleNumber):
    #       z[v].append(solver.IntVar(0, 1, 'z[{}][{}]'.format(v, k)))

    
    Z =  solver.NumVar(0, infinity , 'Z')
    u = []
    for i in range(2*(hanhkhach+goihang)):
      u.append([])
      for k in range(vehicleNumber):
          u[i].append(solver.IntVar(1, 2*(hanhkhach+goihang), 'u[{}][{}]'.format(i, k)))

    r = []
    for i in range(2*(hanhkhach+goihang) +1):
      r.append([])
      for k in range(vehicleNumber):
          r[i].append(solver.IntVar(0,  khoiLuongToiDaMoiXe[k], 'r[{}][{}]'.format(i, k)))
    v = []
    for i in range(2*(hanhkhach+goihang) +1):
      v.append([])
      for k in range(vehicleNumber):
          v[i].append(solver.IntVar(0,  1, 'v[{}][{}]'.format(i, k)))

    q = []
    for i in range(hanhkhach +1):
        q.append(0)
    for i in range(hanhkhach +1 , hanhkhach+goihang +1):
        q.append(khoiLuongGoiHang[i-hanhkhach-1])
    for i in range(hanhkhach+goihang +1 , 2*(hanhkhach)+goihang+1):
        q.append(0)
    for i in range(2*(hanhkhach)+ goihang +1,2*(hanhkhach+goihang)+1):
        q.append(-khoiLuongGoiHang[i-2*(hanhkhach)-goihang-1])

    t = []
    for i in range(hanhkhach +1):
        t.append(1)
    for i in range(hanhkhach +1 , hanhkhach+goihang +1):
        t.append(0)
    for i in range(hanhkhach+goihang +1 , 2*(hanhkhach)+goihang+1):
        t.append(-1)
    for i in range(2*(hanhkhach)+ goihang +1,2*(hanhkhach+goihang)+1):
        t.append(0)
    print('Number of variables =', solver.NumVariables())

    # constrait
    #2.2
    for k in range(vehicleNumber):
      solver.Add(sum(distanceMatrix[i][j]*x[i][j][k]
                    for i in range(2*(hanhkhach+goihang)+1) for j in range(2*(hanhkhach+goihang)+1))  <= Z)




    # 2.3
    for k in range(vehicleNumber):
      solver.Add(sum(x[0][i][k] for i in range(1, 2 * (hanhkhach+goihang)+1)) == 1)
    for k in range(vehicleNumber):
      solver.Add(sum(x[i][0][k] for i in range(1,2 * (hanhkhach+goihang)+1)) == 1)

    # 2.4
    for i in range(1,hanhkhach+goihang +1):
      for k in range(vehicleNumber):
        solver.Add(sum(x[i][j][k]
                    for j in range(2 * (hanhkhach+goihang)+1)) - x[i][i][k] == y[i-1][k])
    for i in range(1,hanhkhach+goihang +1):
      for k in range(vehicleNumber):
        solver.Add(sum(x[j][i][k]
                    for j in range(2 * (hanhkhach+goihang)+1)) - x[i][i][k] == y[i-1][k])

    # 2.5
    for i in range(hanhkhach+goihang+1, 2*(hanhkhach+goihang) + 1):
      for k in range(vehicleNumber):
        solver.Add(sum(x[i][j][k]
                  for j in range(2*(hanhkhach+goihang))) - x[i][i][k] == y[i-hanhkhach-goihang-1][k])
            
    for i in range(hanhkhach+goihang+1, 2*(hanhkhach+goihang) +1):
      for k in range(vehicleNumber):
        solver.Add(sum(x[j][i][k]
                  for j in range(2*(hanhkhach+goihang))) - x[i][i][k] == y[i-hanhkhach-goihang-1][k])

    #2.6
    for i in range(1, 2*(hanhkhach+goihang) +1):
      solver.Add(sum(x[j][i][k] for k in range(vehicleNumber)
                  for j in range(2*(hanhkhach+goihang)+1)) - sum(x[i][i][k] for k in range(vehicleNumber))  == 1)
    # # 2.7
    # for i in range(hanhkhach+goihang):
    #   for k in range(vehicleNumber):
    #     solver.Add(y[i+hanhkhach+goihang][k] >= 1+M*(y[i][k]-1))

    # # 2.8
    # for i in range(hanhkhach+goihang):
    #   for k in range(vehicleNumber):
    #     solver.Add(y[i+hanhkhach+goihang][k] <= 1+M*(1-y[i][k]))

    #2.9
    for i in range(1, hanhkhach+goihang +1):
      for k in range(vehicleNumber):
        solver.Add(u[i-1][k]-u[i+hanhkhach+goihang-1][k]+(2*hanhkhach+2*goihang)*y[i-1][k] <= 2*hanhkhach + 2*goihang-1)
        
    #2.10
    for i in range(1, 2*(hanhkhach+goihang)+1):
      for j in range(1, 2 * (hanhkhach+goihang)+1):
        if j != i:
          for k in range(vehicleNumber):
            solver.Add(u[i-1][k]-u[j-1][k]+(2*hanhkhach+2*goihang)*x[i][j][k] <= 2*hanhkhach + 2*goihang-1)
          
    # #2.11
    # for i in range(2*(hanhkhach+goihang)+1):
    #   for j in range(2 * (hanhkhach+goihang)+1):
    #     for k in range(vehicleNumber):
    #       solver.Add(D[i][k] >= D[j][k] +distanceMatrix[j][i]+M*(x[j][i][k] - 1))
                
    # #2.12
    # for k in range(vehicleNumber):
    #     solver.Add(D[0][k] == 0)
        
    #2.13a
    for i in range(1, hanhkhach+goihang+1):
      for j in range(2 * (hanhkhach+goihang)+1):
        for k in range(vehicleNumber):
          solver.Add(r[i][k] <= r[j][k] +q[i]*y[i-1][k]+ khoiLuongToiDaMoiXe[k] * (1- x[j][i][k]))
    #2.13b
    for i in range(hanhkhach+goihang+1, 2*(hanhkhach+goihang)+1):
      for j in range(2 * (hanhkhach+goihang)+1):
        for k in range(vehicleNumber):
          solver.Add(r[i][k] <= r[j][k] +q[i]*y[i-hanhkhach-goihang-1][k]+ khoiLuongToiDaMoiXe[k] * (1- x[j][i][k]))
          
    #2.14a
    for i in range(1, hanhkhach+goihang+1):
      for j in range(2 * (hanhkhach+goihang)+1):
        for k in range(vehicleNumber):
          solver.Add(r[i][k] >= r[j][k] +q[i]*y[i-1][k] + khoiLuongToiDaMoiXe[k] * (x[j][i][k] - 1))
    #2.14b
    for i in range(hanhkhach+goihang+1, 2*(hanhkhach+goihang)+1):
      for j in range(2 * (hanhkhach+goihang)+1):
        for k in range(vehicleNumber):
          solver.Add(r[i][k] >= r[j][k] +q[i]*y[i-hanhkhach-goihang-1][k] + khoiLuongToiDaMoiXe[k] * (x[j][i][k] - 1))
          
    #2.15
    for i in range(1, 2*(hanhkhach+goihang)+1):
      for k in range(vehicleNumber):
        solver.Add(r[i][k] >= 0)

    for i in range(2*(hanhkhach+goihang)+1):
      for k in range(vehicleNumber):
        solver.Add(r[i][k] <= khoiLuongToiDaMoiXe[k])
        
    #2.16
    for k in range(vehicleNumber):
      solver.Add(r[0][k] == 0)
    
    #2.17a
    for i in range(1, hanhkhach+goihang+1):
      for j in range(2 * (hanhkhach+goihang)+1):
        for k in range(vehicleNumber):
          solver.Add(v[i][k] <= v[j][k] +t[i]*y[i-1][k]+ khoiLuongToiDaMoiXe[k] * (1- x[j][i][k]))
    #2.17b
    for i in range(hanhkhach+goihang+1, 2*(hanhkhach+goihang)+1):
      for j in range(2 * (hanhkhach+goihang)+1):
        for k in range(vehicleNumber):
          solver.Add(v[i][k] <= v[j][k] +t[i]*y[i-hanhkhach-goihang-1][k]+ khoiLuongToiDaMoiXe[k] * (1- x[j][i][k]))
          
    #2.18a
    for i in range(1, hanhkhach+goihang+1):
      for j in range(2 * (hanhkhach+goihang)+1):
        for k in range(vehicleNumber):
          solver.Add(v[i][k] >= v[j][k] +t[i]*y[i-1][k] + khoiLuongToiDaMoiXe[k] * (x[j][i][k] - 1))
    #2.18b
    for i in range(hanhkhach+goihang+1, 2*(hanhkhach+goihang)+1):
      for j in range(2 * (hanhkhach+goihang)+1):
        for k in range(vehicleNumber):
          solver.Add(v[i][k] >= v[j][k] +t[i]*y[i-hanhkhach-goihang-1][k] + khoiLuongToiDaMoiXe[k] * (x[j][i][k] - 1))
          
    #2.20
    for k in range(vehicleNumber):
      solver.Add(v[0][k] == 0)

    #2.21
    for i in range(2*(hanhkhach+goihang)+1):
      for k in range(vehicleNumber):
        solver.Add(x[i][i][k] == 0)
    print('Number of constraints =', solver.NumConstraints())

    solver.Minimize(Z)

    status=solver.Solve()
    # print(solver.ExportModelAsLpFormat(False).replace('\\', '').replace(',_', ','), sep='\n')
    f = open(r"/content/drive/MyDrive/Tối ưu/data/DataSet{}/output.txt".format(count),"w+")
    if status == pywraplp.Solver.FEASIBLE or status == pywraplp.Solver.OPTIMAL :
        print('Solution:')
        print('Objective value =', solver.Objective().Value())
        f.writelines("Objective value = {}\n".format(solver.Objective().Value()))
        for k in range(vehicleNumber):
          print("-- Vehicle {} --".format(k))
          l = []
          start = 0
          while True:
            l.append(start)
            print("Point {}, capacity: {}, number of passenger: {}".format(start, r[start][k].solution_value(), v[start][k].solution_value()))
            for i in range(2*(hanhkhach+goihang)+1):
              if x[start][i][k].solution_value() == 1:
                start = i
                break
            if i == 0:
              break
          print("++ Visiting order: {} ++".format(l))
          f.writelines("Taxi {0} visit {1}\n".format(k,l))
          s = distanceMatrix[l[-1]][0]
          for i in range(1, len(l)):
            s += distanceMatrix[l[i-1]][l[i]]
          print("++ Distance: {} ++".format(s))
    
    else:
        print('The problem does not have an optimal solution.')

    print('\nAdvanced usage:')
    f.writelines('Problem solved in %f milliseconds\n' % solver.wall_time())

    print('Problem solved in %f milliseconds' % solver.wall_time())
    f.writelines('Problem solved in %d iterations\n' % solver.iterations())
    print('Problem solved in %d iterations' % solver.iterations())
    f.writelines('Problem solved in %d branch-and-bound nodes\n' % solver.nodes())
    print('Problem solved in %d branch-and-bound nodes' % solver.nodes())
    f.close()


if __name__ == '__main__':
  main()

Number of variables = 2566
Number of constraints = 10905
Solution:
Objective value = 217.99999999999994
-- Vehicle 0 --
Point 0, capacity: 0.0, number of passenger: 0.0
Point 10, capacity: 23.0, number of passenger: 0.0
Point 20, capacity: 0.0, number of passenger: 0.0
++ Visiting order: [0, 10, 20] ++
++ Distance: 204 ++
-- Vehicle 1 --
Point 0, capacity: 0.0, number of passenger: 0.0
Point 3, capacity: 0.0, number of passenger: 1.0
Point 13, capacity: 0.0, number of passenger: 0.0
++ Visiting order: [0, 3, 13] ++
++ Distance: 157 ++
-- Vehicle 2 --
Point 0, capacity: 0.0, number of passenger: 0.0
Point 4, capacity: 0.0, number of passenger: 1.0
Point 9, capacity: 23.0, number of passenger: 1.0
Point 14, capacity: 23.0, number of passenger: 0.0
Point 1, capacity: 23.0, number of passenger: 1.0
Point 11, capacity: 23.0, number of passenger: 0.0
Point 19, capacity: 0.0, number of passenger: 0.0
++ Visiting order: [0, 4, 9, 14, 1, 11, 19] ++
++ Distance: 213 ++
-- Vehicle 3 --
Point 0, c